In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [97]:
!wget http://iali.in/datasets/Social_Network_Ads.csv
!pip install flask-ngrok

--2020-06-07 17:57:44--  http://iali.in/datasets/Social_Network_Ads.csv
Resolving iali.in (iali.in)... 156.67.222.125
Connecting to iali.in (iali.in)|156.67.222.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10926 (11K) [text/plain]
Saving to: ‘Social_Network_Ads.csv.4’

Social_Network_Ads. 100%[===================>]  10.67K  --.-KB/s    in 0s      

2020-06-07 17:57:45 (220 MB/s) - ‘Social_Network_Ads.csv.4’ saved [10926/10926]



In [98]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [0]:
x = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [0]:
from sklearn import tree
clf=tree.DecisionTreeClassifier()
clf=clf.fit(x_train,y_train)

In [102]:
y_pred=clf.predict(np.array([[39,  61000]]))
y_pred

array([0])

In [0]:
pickle.dump(clf, open('clfmodel.pkl','wb'))

In [104]:
model = pickle.load(open('clfmodel.pkl','rb'))
print(model.predict([[39,  61000]]))

[0]


In [0]:
from flask import Flask, request, jsonify, render_template
app = Flask(__name__)
run_with_ngrok(app)
model = pickle.load(open('clfmodel.pkl', 'rb'))

In [0]:
@app.route('/')
def home():
    return render_template('index.html')


In [0]:
@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [int(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = model.predict(final_features)

    output = round(prediction[0], 2)
    if output==1.0:
      stroutput="Will Purchase"
    else:
      stroutput="Will not Purchase"

    return render_template('index.html', prediction_text='Status of purchase : {}'.format(stroutput))

In [0]:
@app.route('/predict_api',methods=['POST'])
def predict_api():
    '''
    For direct API calls trought request
    '''
    data = request.get_json(force=True)
    prediction = model.predict([np.array(list(data.values()))])

    output = prediction[0]
    return jsonify(output)

In [109]:
from flask_ngrok import run_with_ngrok
import traceback
try:
  app.run()
except:
  print("hi")
  traceback.print_exc()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://b0228cf21c45.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jun/2020 17:58:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 17:58:19] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 17:58:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2020 17:58:27] "POST /predict HTTP/1.1" 200 -
